In [ ]:
!pip install -r ../requirements.txt

In [2]:
# !pip install pyrdf2vec

In [3]:
import random
import os
import numpy as np
import rdflib
import pandas as pd
import matplotlib.pyplot as plt
import shutil

from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.manifold import TSNE

from rdf2vec.converters import rdflib_to_kg
from rdf2vec.walkers import RandomWalker
from rdf2vec import RDF2VecTransformer

import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'numpy'

## Import the rdf file (ttl, nt, all other supported by rdflib)

In [ ]:
url = 'https://raw.githubusercontent.com/MaastrichtU-IDS/neuro_dkg/master/data/output/neuro_dkg.ttl'
rdf_file ='neurodkg.ttl'
# rdf_file = url.split('/')[-1]
#rdf_file = 'input/covid19-literature-knowledge-graph/sample_kg.nt'
#fileext = '.nq.gz'

# Download the RDF file
with requests.get(url, stream=True) as r:
    with open(rdf_file, 'wb') as f:
        r.raw.read = functools.partial(r.raw.read, decode_content=True)
        shutil.copyfileobj(r.raw, f)

#predicates for Random Walker to follow
label_predicates = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>']

In [8]:
kg = rdflib_to_kg(rdf_file, filetype='turtle')

100%|██████████| 1420/1420 [00:00<00:00, 44074.44it/s]


In [9]:
# We'll all possible walks of depth 2
random_walker = RandomWalker(2, 4)

# Create embeddings with random walks
transformer = RDF2VecTransformer(walkers=[random_walker], sg=1)


<SparkContext master=local[10] appName=pyspark-shell>


In [10]:
all_entities = kg.get_all_entities()

In [11]:
all_entities[:10]

['http://www.w3id.org/neurodkg/Instances/context96',
 'Obsessive Compulsive Disoder',
 'http://www.w3id.org/neurodkg/Instances/context65',
 'Bipolar I disorder',
 'Alprazolam',
 'http://www.w3id.org/neurodkg/Instances/TargetGroup93',
 'moderate dementia',
 'minor arthritis pain',
 'depressive episode',
 'http://purl.obolibrary.org/obo/MFOMD_0000143']

In [12]:
walk_embeddings = transformer.fit_transform(kg, all_entities)

./walks/randwalks_n4_depth2_pagerank_uniform.txt
Time elapsed to generate features: 00:00:03
Extracted 0 walks for 966 instances!
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform
Processing  uniform


In [13]:
walk_embeddings[:10]

[array([ 0.03558761, -0.04060734, -0.00455666, -0.11629998,  0.08953823,
        -0.06534512,  0.05720286,  0.03025358, -0.11612691,  0.05921733,
         0.0422963 ,  0.00074582,  0.00835328, -0.00833152,  0.0621771 ,
         0.05002274,  0.00948131, -0.00041636,  0.04247904, -0.05160643,
        -0.01788511, -0.06088221,  0.12318227,  0.04710478,  0.07801736,
        -0.05029538,  0.05629896,  0.03515671,  0.00568493, -0.10584523,
         0.07312205, -0.09889383, -0.07533701, -0.02709199,  0.00327425,
         0.05302596, -0.00913332, -0.06664476, -0.04088874,  0.01206832,
         0.00786199,  0.03949703, -0.03144273, -0.0526722 ,  0.11196816,
         0.00241974, -0.01913813,  0.05643567,  0.02105749, -0.01069479,
        -0.00872755,  0.09823257,  0.03824679,  0.02306523, -0.03236259,
        -0.00541069,  0.04079197, -0.03703165,  0.01105529,  0.02898223,
         0.08056394,  0.04600153, -0.00868432,  0.10156506,  0.05411338,
        -0.03275768, -0.02767232, -0.00905999, -0.0

In [14]:
len(all_entities)

966

In [15]:
len(walk_embeddings)

966

## Generating a dataframe for entity embeddings

In [16]:
df =pd.DataFrame(zip(all_entities, walk_embeddings), columns=['entity', 'embedding'])
    

In [17]:
# a function for converting entity names
# if you need to provide entity names with CURIE format (e.g. DRUGBANK:DB00012)
def replace_prefix(entity):
    if entity.startswith('http://www.w3id.org/drugbank:'):
        return entity.replace('http://www.w3id.org/drugbank:', 'DRUGBANK:')
    else:
        return entity

df.entity = df.entity.apply(replace_prefix)

### Convert dataframe embeddings to JSON

And store the dataframe in a JSON file, to be imported in OpenPredict

In [18]:
df.to_json('neurodkg_embedding.json',orient='records')

In [19]:
import pandas as pd
import numpy as np
df =pd.read_json('neurodkg_embedding.json',orient='records')

In [20]:
df.head()

,entity,embedding
0,http://www.w3id.org/neurodkg/Instances/context96,"[0.035587612500000004, -0.0406073444, -0.00455..."
1,Obsessive Compulsive Disoder,"[0.0008083943, -0.0005201186000000001, 0.00010..."
2,http://www.w3id.org/neurodkg/Instances/context65,"[0.0303201601, -0.0355585478, -0.0045577516, -..."
3,Bipolar I disorder,"[0.0009355723000000001, -0.0005682872000000001..."
4,Alprazolam,"[-0.0007071238, 0.0002212713, 0.00059985950000..."


In [34]:
print(df['entity'])
# np.array(df['embedding'].values)

0       http://www.w3id.org/neurodkg/Instances/context96
1                           Obsessive Compulsive Disoder
2       http://www.w3id.org/neurodkg/Instances/context65
3                                     Bipolar I disorder
4                                             Alprazolam
                             ...                        
961                        http://www.w3id.org/doid/2377
962    http://www.w3id.org/neurodkg:hasCurrentMedication
963     http://www.w3id.org/neurodkg/Instances/context67
964                                           Lurasidone
965    http://purl.bioontology.org/ontology/MEDDRA/10...
Name: entity, Length: 966, dtype: object


In [25]:
embedding_mat =[]
for i, row in df.iterrows():
    emb=row['embedding']
    embedding_mat.append(emb)

In [26]:
entities = df.entity.to_list()

### alternatively you can store as csv with X columns (X is the dimension of the emebdding)

In [27]:
df_emb =pd.DataFrame( embedding_mat, columns= ['feature'+str(i) for i in range(len(emb))])

In [28]:
df_emb['entity'] = entities

In [29]:
df_emb.to_csv('neurodkg_embedding.csv', index=False)

In [30]:
df_emb.head()

,feature0,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,...,feature491,feature492,feature493,feature494,feature495,feature496,feature497,feature498,feature499,entity
0,0.035588,-0.040607,-0.004557,-0.116300,0.089538,-0.065345,0.057203,0.030254,-0.116127,0.059217,...,-0.075535,-0.028785,-0.067564,-0.027185,0.021131,-0.012922,-0.002191,-0.006943,0.050208,http://www.w3id.org/neurodkg/Instances/context96
1,0.000808,-0.000520,0.000105,0.000219,0.000143,0.000348,0.000566,-0.000963,-0.000512,0.000226,...,0.000967,-0.000680,0.000385,0.000655,0.000958,0.000648,-0.000814,-0.000580,-0.000469,Obsessive Compulsive Disoder
2,0.030320,-0.035559,-0.004558,-0.100596,0.076648,-0.057881,0.050034,0.027175,-0.099821,0.050196,...,-0.065656,-0.025718,-0.058742,-0.023038,0.018239,-0.011436,-0.001219,-0.006044,0.043053,http://www.w3id.org/neurodkg/Instances/context65
3,0.000936,-0.000568,0.000017,-0.000726,0.000052,0.000546,0.000048,0.000370,0.000012,0.000068,...,-0.000888,0.000442,-0.000057,0.000538,-0.000321,0.000910,0.000044,-0.000649,0.000515,Bipolar I disorder
4,-0.000707,0.000221,0.000600,0.000540,0.000113,-0.000533,0.000437,-0.000432,-0.000605,0.000940,...,0.000640,-0.000752,0.000320,-0.000095,0.000579,0.000074,-0.000232,-0.000223,0.000047,Alprazolam
